In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from numpy import array

2023-02-02 19:01:42.615687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 19:01:44.236241: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-02 19:01:47.097547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-02-02 19:01:47.098380: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
la_2020 = pd.read_csv('./data/LA_pm10_2020.csv')
la_2021 = pd.read_csv('./data/LA_pm10_2021.csv')
la_2022 = pd.read_csv('./data/LA_pm10_2022.csv')

In [4]:
# subsetting
la_2020 = la_2020[['Date', 'Daily Mean PM10 Concentration']]
la_2021 = la_2021[['Date', 'Daily Mean PM10 Concentration']]
la_2022 = la_2022[['Date', 'Daily Mean PM10 Concentration']]

In [5]:
# converting all to datetime
la_2020['Date'] = pd.to_datetime(la_2020['Date'])
la_2021['Date'] = pd.to_datetime(la_2021['Date'])
la_2022['Date'] = pd.to_datetime(la_2022['Date'])

In [6]:
# merging
la_pm10 = pd.concat([la_2020, la_2021, la_2022])
la_pm10.rename(columns = {'Daily Mean PM10 Concentration':'daily_pm10'}, inplace = True)

In [7]:
#merging
la_pm10 = pd.concat([la_2020, la_2021])
la_pm10.rename(columns = {'Daily Mean PM10 Concentration':'daily_pm10'}, inplace = True)
la_2022.rename(columns = {'Daily Mean PM10 Concentration':'daily_pm10'}, inplace = True)

In [8]:
# dropping null cols
la_pm10 = la_pm10.dropna()

In [9]:
la_pm10.head()

,Date,daily_pm10
0,2020-01-04,25
1,2020-01-10,20
2,2020-01-16,32
3,2020-01-22,21
4,2020-01-28,23


In [10]:
la_pm10.tail()

,Date,daily_pm10
983,2021-12-27,6
984,2021-12-28,6
985,2021-12-29,5
986,2021-12-30,4
987,2021-12-31,7


In [11]:
# finding row with max pm10 value
la_pm10.loc[(la_pm10['daily_pm10'] == max(la_pm10['daily_pm10']))]

,Date,daily_pm10
906,2021-10-11,411


In [12]:
#normalizing
la_pm10['daily_pm10'] = (la_pm10['daily_pm10'] - la_pm10['daily_pm10'].mean()) / la_pm10['daily_pm10'].std()

In [13]:
la_pm10

,Date,daily_pm10
0,2020-01-04,-0.159737
1,2020-01-10,-0.422918
2,2020-01-16,0.208717
3,2020-01-22,-0.370282
4,2020-01-28,-0.265009
...,...,...
983,2021-12-27,-1.159826
984,2021-12-28,-1.159826
985,2021-12-29,-1.212462
986,2021-12-30,-1.265098


Data Split

In [14]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [15]:
# define input sequence
daily_pm10 = la_pm10['daily_pm10'].values.tolist()

In [16]:
daily_pm10 = daily_pm10[0:len(daily_pm10)-1]

In [17]:
x_train, y_train = split_sequence(daily_pm10, 10)


In [18]:
x_train

array([[-0.15973694, -0.42291816,  0.20871675, ...,  0.20871675,
        -0.05446446, -0.4755544 ],
       [-0.42291816,  0.20871675, -0.37028191, ..., -0.05446446,
        -0.4755544 ,  0.36662548],
       [ 0.20871675, -0.37028191, -0.26500943, ..., -0.4755544 ,
         0.36662548, -1.21246179],
       ...,
       [-0.4755544 , -0.26500943,  0.10344427, ..., -1.21246179,
        -1.26509804, -1.15982555],
       [-0.26500943,  0.10344427,  0.261353  , ..., -1.26509804,
        -1.15982555, -1.15982555],
       [ 0.10344427,  0.261353  ,  0.10344427, ..., -1.15982555,
        -1.15982555, -1.21246179]])

In [19]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))


In [20]:
x_train.shape

(1918, 10, 1)

In [21]:
# define model
n_steps = 10
n_features = 1
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

2023-02-02 19:04:41.622919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-02 19:04:41.867408: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-02 19:04:41.868050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-02 19:04:41.872682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [22]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [24]:
model.fit(x_train, y_train, epochs=200, verbose=0)

2023-02-02 19:05:35.221126: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [28]:
import joblib
from joblib import Parallel, delayed

joblib.dump(model, './models/keras_lstm_1.pkl')

INFO:tensorflow:Assets written to: ram://038dc3c3-4b9e-4d96-a99f-13866720d6b2/assets


['./models/keras_lstm_1.pkl']

In [26]:
x_train[0]

array([[-0.15973694],
       [-0.42291816],
       [ 0.20871675],
       [-0.37028191],
       [-0.26500943],
       [-0.15973694],
       [-0.52819064],
       [ 0.20871675],
       [-0.05446446],
       [-0.4755544 ]])

In [29]:
model = joblib.load('./models/keras_lstm_1.pkl')